In [ ]:
# %load make_subj_csvs.py
#!/usr/bin/env python2
"""
Created on Mon Jul 16 14:13:15 2018

For every subject, make csvs with all the info relevant for generating walks for their experiment

This includes: 
    image_mapping.csv - mapping from nodes to pngs
    prac_walk - practice order with images
    exposure_walk - full task with images

@author: stiso
"""
import random
import csv
import os
import math

nNode = 10
nSubj = 60
nSess = 3
nTrial = 1001

# Key presses
keys = ['q', 'w', 'e', 'r', 'v', 'b', 'u', 'i', 'o', 'p']

# create stimuli png list
stim = []
stim_prac = []
resp = [0,1,2,3,4,5,6,7,8,9]
for i in range(nNode):
    stim.append("".join(('img/target_', str(resp[i]+1), '.png')))
    stim_prac.append("".join(('img/target_', str(resp[i]+1), '_prac.png')))

In [ ]:
    
for i in range(nSubj):
    # save dir: consistent with Steve's naming scheme
    save_dir = "".join(['/Users/stiso/Documents/Python/graphLearning/subj', str(i+1), '/'])
    if not os.path.exists(save_dir):
            os.makedirs(save_dir)
    else:
        print("You have already made these files! delete them first and rerun")
        break
            
    # add first row of all CSVs
    for s in range(nSess):
        csvfile = "".join([save_dir, 'exposure_walk{}.csv'.format(str(s+1))])
        with open(csvfile, "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            wr.writerow(('pID', 'sess', 'trialNum', 'graph', 'walk', 'resp','path'))
    csvfile = "".join([save_dir, 'image_mapping.csv'])
    with open(csvfile, "a") as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(('sID','imageNums', 'imageFilenames'))
    csvfile = "".join([save_dir, 'prac_walk.csv'])
    with open(csvfile, "a") as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(('pID', 'trialNum', 'walk', 'resp','path'))
    
    # create random mapping: randomize list of 10 stimuli
    all_stim = list(zip(stim,keys,stim_prac))
    random.shuffle(all_stim)
    stim,keys,stim_prac = zip(*all_stim)
    
    # write to csv
    csvfile = "".join([save_dir, 'image_mapping.csv'])
    sID = [i+1]*nNode
    all_mapping = [sID, range(1, nNode+1), stim]
    l = zip(*all_mapping)
    with open(csvfile, "a") as fp:
        wr = csv.writer(fp, dialect='excel')
        for entry in l:
            wr.writerow(entry)
        
    # practice trials just go 1:10 for now
    trialNum = range(1, nNode+1)
    walk = range(1, nNode+1)
    
    # save to csv
    all_prac = [sID, trialNum, walk, keys, stim_prac]
    l = zip(*all_prac)
    csvfile = "".join([save_dir, 'prac_walk.csv'])
    with open(csvfile, "a") as fp:
        wr = csv.writer(fp, dialect='excel')
        for entry in l:
            wr.writerow(entry)
    
    # make walks
    if i%2 == 0:
        
        # if even, use ring lattice
        if nNode == 10:
            filename = "/Users/stiso/Documents/Python/graphLearning/ring_lattice10.csv"
        else:
            filename = "/Users/stiso/Documents/Python/graphLearning/ring_lattice.csv"
        graph = ['lattice']
            
    else:
       # if odd, use modular
        if nNode == 10:
            filename = "/Users/stiso/Documents/Python/graphLearning/modular10.csv"
        else:
            filename = "/Users/stiso/Documents/Python/graphLearning/modular.csv"
        graph = ['modular']
    
    
    with open(filename) as f:
        reader = csv.reader(f)
        cnt = 0
        for row in reader:
            if cnt == i:
                walks = row
                break
            cnt += 1
    # if walks is a list of 1, break up at semi colon
    if len(walks) == 1:
        walks = walks[0].split(';')
        
    # get mapping to image files and responses
    full_stim = [stim[int(j)-1] for j in walks]
    full_resp = [keys[int(j)-1] for j in walks]
        
    # write to csv
    sID = [i+1]*nTrial
    graph = graph*nTrial
    
    # break up data by session
    for s in range(nSess):
        #import pdb; pdb .set_trace()
        # get index for each session
        st = s*(nTrial-1)
        en = st+nTrial

        sess = [s+1]*nTrial
        all_exp = [sID, sess, range(1,nTrial), graph, walks[st:en], full_resp[st:en], full_stim[st:en]]
        l = zip(*all_exp)
        csvfile = "".join([save_dir, 'exposure_walk{}.csv'.format(s+1)])
        with open(csvfile, "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            for entry in l:
                wr.writerow(entry)